# Temat: "Deskryptory / Dekoratory / Właściwości"
### Imię nazwisko: Eduard Povierin
### Grupa: 2TD12A

---

## Zadanie #2

In [1]:
# Porównać działanie poniższego kodu i kodu z przykładu 4. Omówić efekt wykorzystania dekoratora @wraps.
from functools import wraps

def my_decorator(f):
    @wraps(f) # <- dodajemy dekorator
    def wrapper(*args, **kwds):
        """Wraper docstring"""
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper


@my_decorator
def example():
    """Docstring"""
    print('Called example function')

example()
print(example.__name__)
print(example.__doc__)

Calling decorated function
Called example function
example
Docstring


## Zadanie #3

In [7]:
# Utworzyć własne przykłady wykorzystujące deskryptory.
class Celsius:
    def __init__(self, temperature=0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    def get_temperature(self):
        print("Getting value...")
        return self._temperature

    def set_temperature(self, value):
        if value < -273:
            raise ValueError("Temperatura poniżej -273 nie jest możliwa")
        print("Setting value...")
        self._temperature = value

    temperature = property(get_temperature, set_temperature)

c = Celsius()
c.temperature = 37
print(c.temperature)
print("{:.2f}".format(c.to_fahrenheit()))


Setting value...
Setting value...
Getting value...
37
Getting value...
98.60


## Zadanie #4

In [9]:
# Utworzyć własne przykłady wykorzystujące dekoratory.
from datetime import datetime

def log(func):
    def wrapper(*args, **kwargs):
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        args_str = ', '.join([str(arg) for arg in args])
        kwargs_str = ', '.join([f'{key}={value}' for key, value in kwargs.items()])
        all_args_str = ', '.join(filter(None, [args_str, kwargs_str]))
        print(f'{timestamp} - {func.__name__}({all_args_str})')
        result = func(*args, **kwargs)
        print(f'{timestamp} - {func.__name__} returned {result}')
        return result
    return wrapper

@log
def add_numbers(a, b):
    return a + b

print(add_numbers(3, 5)) # Output: 8


2023-11-01 14:41:40 - add_numbers(3, 5)
2023-11-01 14:41:40 - add_numbers returned 8
8


## Zadanie #5

In [15]:
# Napisać własny dekorator i zaprezentować jego działanie.

# Importujemy moduł time, który pozwoli nam zmierzyć czas wykonania funkcji
import time

# Definiujemy dekorator measure_time, który przyjmuje funkcję jako argument
def measure_time(func):
    # Definiujemy wewnętrzną funkcję wrapper, która będzie wykonywać naszą funkcję i mierzyć czas jej wykonania
    def wrapper(*args, **kwargs):
        # Zapisujemy czas rozpoczęcia wykonywania funkcji
        start_time = time.time()
        # Wykonujemy funkcję i zapisujemy jej wynik
        result = func(*args, **kwargs)
        # Zapisujemy czas zakończenia wykonywania funkcji
        end_time = time.time()
        # Wypisujemy na ekranie czas wykonania funkcji
        print(f"Function {func.__name__} took {end_time - start_time:.5f} seconds to execute.")
        # Zwracamy wynik funkcji
        return result
    # Zwracamy wewnętrzną funkcję wrapper
    return wrapper


# Dekorujemy funkcję add_numbers dekoratorem measure_time
@measure_time
def add_numbers_timed(a, b):
    time.sleep(1)
    return a + b

# Wywołujemy funkcję add_numbers_timed z argumentami 3 i 5
print(add_numbers_timed(3, 5))


Function add_numbers_timed took 1.00375 seconds to execute.
8


## Zadanie #6

In [16]:
# Utworzyć własne przykłady wykorzystujące funkcję property().
class Person:
    def __init__(self, name, age):
        self.name = name
        self._age = age

    def get_age(self):
        return self._age

    def set_age(self, age):
        if age < 0:
            raise ValueError("Age cannot be negative")
        self._age = age

    age = property(get_age, set_age)

p = Person("Alice", 25)
print(p.age) # 25
p.age = 30
print(p.age) # 30


25
30


## Zadanie #7

In [17]:
# Utworzyć własne przykłady wykorzystujące dekorator property().
class Rectangle:
    def __init__(self, length, width):
        self.length = length
        self.width = width

    @property
    def area(self):
        return self.length * self.width

    @property
    def perimeter(self):
        return 2 * (self.length + self.width)

r = Rectangle(5, 3)
print(r.area) # 15
print(r.perimeter) # 16


15
16


## Dziękuję za uwagę!